# Modelamiento no supervisado en base a tópicos

In [1]:
import sys
sys.path.insert(0, '..')
from utils.preprocesamiento import StemmerTokenizer

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

In [2]:
from utils.cargar import df_caso
from utils.preprocesamiento import process_df, procesar_adela

caso = 'alicia'
df = df_caso(caso)

df = process_df(df,'comment','sel',verbose=True)

df = df.drop(columns=['user_id','team_id','gender','df','title','opt_left','opt_right','max_num','phase','time','curso'])

80 rows found with non string elements for column comment (0.65%)
Deleting 2326 columns for which max target value is over 7 (18.76%)
9991 available rows after processing


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_indices, test_indices = train_test_split(range(len(df)), test_size=.05, stratify=df['sel'], random_state=0)

try:
    train_indices = pd.read_csv('../utils/splits/train_indices_{}.csv'.format(caso), header=None)[0].tolist()
    test_indices = pd.read_csv('../utils/splits/test_indices_{}.csv'.format(caso), header=None)[0].tolist()
    print("Partición train-test cargada")
except FileNotFoundError:
    pd.Series(train_indices).to_csv('../utils/splits/train_indices_{}.csv'.format(caso), index=False)
    pd.Series(test_indices).to_csv('../utils/splits/test_indices_{}.csv'.format(caso), index=False)
    print("Partición train-test guardada")

df_train = df.iloc[train_indices]
df_test = df.iloc[test_indices]

Partición train-test guardada


In [4]:
tokenized_corpus = [tokenizador(document) for document in df_train['comment']]
tokenized_test = [tokenizador(document) for document in df_test['comment']]

In [5]:
import gensim
from gensim import corpora

# Create a dictionary from the tokenized corpus
dictionary = corpora.Dictionary(tokenized_corpus)

# Convert the tokenized corpus into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_corpus]

Usamos el número de tópicos con mejor métrica de coherencia

In [6]:
%%time
lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=6, id2word=dictionary, passes=10)

CPU times: user 22.8 s, sys: 2.31 ms, total: 22.8 s
Wall time: 22.9 s


In [7]:
# Print the generated topics
topics = lda_model.print_topics(num_topics=6)
for topic in topics:
    print(topic)

(0, '0.041*"reputación" + 0.035*"transparencia" + 0.022*"usuarios" + 0.018*"mantener" + 0.017*"importante" + 0.015*"profesional" + 0.014*"mandante" + 0.014*"ser" + 0.013*"si" + 0.013*"personal"')
(1, '0.055*"proyecto" + 0.021*"entregar" + 0.018*"criterios" + 0.015*"priorizar" + 0.015*"técnicos" + 0.014*"usuarios" + 0.014*"si" + 0.013*"plazo" + 0.013*"Alicia" + 0.013*"plazos"')
(2, '0.034*"ser" + 0.024*"si" + 0.016*"proyecto" + 0.015*"transparente" + 0.014*"Alicia" + 0.011*"podría" + 0.010*"reputación" + 0.008*"usuario" + 0.008*"producto" + 0.007*"empresa"')
(3, '0.025*"si" + 0.021*"mas" + 0.020*"proyecto" + 0.011*"tiempo" + 0.009*"podria" + 0.009*"problemas" + 0.009*"puede" + 0.008*"reputacion" + 0.008*"asi" + 0.007*"usuario"')
(4, '0.036*"grupo" + 0.021*"postura" + 0.015*"anterior" + 0.013*"2" + 0.013*"opinión" + 0.011*"1" + 0.011*"respuesta" + 0.009*"acuerdo" + 0.009*"Mi" + 0.009*"3"')
(5, '0.026*"contexto" + 0.026*"cumplir" + 0.020*"acuerdos" + 0.018*"proyecto" + 0.017*"debe" + 0.01

In [8]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
pyLDAvis.display(vis_data)

In [9]:
print(' '.join(tokenized_test[0]) + '\n')

# Convert the tokenized document into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(tokenized_test[0])]

# Get the topic probabilities for the new document
topic_probs = lda_model.get_document_topics(doc_term_matrix)[0]

# Print the topic probabilities
for topic, prob in topic_probs:
    print(f"Topic {topic}: {prob}")

Es importante producir producto aceptable incluso si demora pues hacer producto mal puede llevar repercuciones grandes

Topic 1: 0.8954811692237854
Topic 3: 0.010026677511632442
Topic 4: 0.06456241011619568
Topic 5: 0.010011440142989159


## Correlaciones

In [10]:
from TM_utils import get_lda_embeddings
import pandas as pd

arr_test = get_lda_embeddings(lda_model,tokenizador,dictionary,df_test,'comment')
df_topics = pd.DataFrame(arr_test, columns=['Topic{}'.format(i+1) for i in range(len(topics))])

df_topics['sel'] = df_test['sel'].values

In [11]:
from scipy.stats import pearsonr

# Only compute pearson prod-moment correlations between feature
# columns and target column
target_col_name = 'sel'
feature_target_corr = {}
for col in df_topics:
    if target_col_name != col:
        feature_target_corr[col + '_' + target_col_name] = \
            pearsonr(df_topics[col], df_topics[target_col_name])[0]
print("Feature-Target Correlations")
print(feature_target_corr)

Feature-Target Correlations
{'Topic1_sel': -0.28861264907086626, 'Topic2_sel': -0.16211091091260782, 'Topic3_sel': -0.186443023745536, 'Topic4_sel': -0.04149402264454165, 'Topic5_sel': 0.06290618829714725, 'Topic6_sel': 0.5377906254167664}
